In [1]:
# Imports
from datetime import datetime
import numpy as np
import pandas as pd

In [2]:
# Define constants

# See https://aqs.epa.gov/aqsweb/documents/codetables/states_and_counties.html
STATE_CODE = 41 # Oregon
COUNTY_CODE = 39 # Lane County, OR

# Data source: https://aqs.epa.gov/aqsweb/airdata/download_files.html
# PM2.5 - Local Conditions: 88101
# PM10 Total 0-10um STP: 81102
# Nitrogen dioxide (NO2): 42602
# Sulfur dioxide: 42401
# Ozone: 44201
# See https://aqs.epa.gov/aqsweb/documents/codetables/methods_criteria.html


In [3]:
# Load data
county_aqi = pd.read_csv("input_data/daily_aqi_by_county_2022.csv")
PM_2pt5 = pd.read_csv("input_data/daily_88101_2022.csv")
PM_10 = pd.read_csv("input_data/daily_81102_2022.csv")
NO2 = pd.read_csv("input_data/daily_42602_2022.csv")
SO2 = pd.read_csv("input_data/daily_42401_2022.csv")
O3 = pd.read_csv("input_data/daily_44201_2022.csv")

# Create list of data
data_list = [PM_2pt5, PM_10, NO2, SO2, O3]
suffix_list = ["PM_2pt5", "PM_10", "NO2", "SO2", "O3"]

/var/folders/bx/8_p36bpd14nckr66rq2kk6kc0000gn/T/ipykernel_15083/1768561378.py:3: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  PM_2pt5 = pd.read_csv("input_data/daily_88101_2022.csv")


In [4]:
# Filter state and county for AQI dataset
county_aqi_query = county_aqi.query(f"`State Code` == {STATE_CODE} and `County Code` == {COUNTY_CODE}")

#COL_NAMES = ["Site Num", "Parameter Code", "Latitude", "Longitude", "Datum", "Parameter Name", "Date Local", "Units of Measure", "Arithmetic Mean"]
COL_NAMES = ["Address", "Parameter Name", "Date Local", "Units of Measure", "Arithmetic Mean"]

def filter_df(df):
    # Filter data for the state and county of interest
    filter_location = df.query(f"`State Code` == {STATE_CODE} and `County Code` == {COUNTY_CODE} and `Sample Duration` == '24-HR BLK AVG'")
    # Select columns of interest
    filter_columns = filter_location.loc[:, COL_NAMES]
    
    return filter_columns

# Concatenate filtered data frames
filtered_concat = pd.concat(list(map(filter_df, data_list)))

# Join filtered data to AQI data
county_aqi_query = county_aqi_query.merge(right=filtered_concat, left_on="Date", right_on="Date Local", suffixes = ("", "_params"))



In [80]:
# Check output concatenated data
#filtered_concat[filtered_concat["Parameter Name"] == "PM2.5 - Local Conditions"]
#county_aqi.head()

# Reshape data to get multiple parameters for each address and date
aqi_pivot = county_aqi_query.pivot(index=["Date", "Address"], columns="Parameter Name", values=["AQI", "Arithmetic Mean"])
# Remove rows with null values
aqi_clean = aqi_pivot.dropna()
#aqi_clean.columns.to_flat_index()
aqi_clean = aqi_clean.droplevel(level=0, axis=1)
# Rename columns
#aqi_clean.columns
aqi_clean.columns = ["AQI", "AQI1", "PM10", "PM2.5"]
# Get data rows
aqi_clean = aqi_clean.iloc[2:,0:]
# Flatten multi-index columns
#aqi_clean.columns = aqi_clean.columns.map('_'.join)
# Drop duplicate AQI column
aqi_clean.drop(columns=["AQI1"], inplace=True)



In [81]:
# Save data to csv
aqi_clean.to_csv("clean_data/aqi.csv")